# Dependence

In [ ]:
import torch
import torch.nn as nn
import torch.autograd as autograd

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.tri as mtri

seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Parameter

In [ ]:
steps = 10000
layers = np.array([2,32,64,32,2])
lr = 0.001

# Build mesh

In [ ]:
m_rows = 20
m_cols = 20

x = torch.linspace(0, 1, m_cols + 1)
y = torch.linspace(0, 1, m_rows + 1)
x = x.repeat(m_rows + 1)
y = y.repeat_interleave(m_cols + 1)
V = torch.stack([x,y],dim=1)
V_ref = V.clone()

tris = []

for i in range(m_cols):
   for j in range(m_rows):
       tris.append([j * (m_cols + 1) + i, j * (m_cols + 1) + i + 1, (j + 1) * (m_cols + 1) + i + 1])
       tris.append([j * (m_cols + 1) + i, (j + 1) * (m_cols + 1) + i + 1, (j + 1) * (m_cols + 1) + i])
        
tris = torch.tensor(tris)
boundarys = V[(V[:,0]==0.0) | (V[:,1]==0.0) | (V[:,0]==1.0) | (V[:,1]==1.0)]

plt.triplot(x,y,tris,'k-',linewidth=0.3)
plt.show()

# Moving Mesh Functions

In [ ]:
def U(V):
    return torch.tanh(-30*(V[:,1]-0.5-0.25*torch.sin(2*torch.pi*V[:,0])))

def U_x_y(V):
    U_x = (1-U(V)**2)*(15*torch.pi*torch.cos(2*torch.pi*V[:,0]))
    U_y = (1-U(V)**2)*(-30)
    return torch.stack([U_x,U_y],dim=1)
    
def metric(V):
    u_x_y = U_x_y(V)
    col = u_x_y.unsqueeze(2)
    row = u_x_y.unsqueeze(1)
    u_x_y_2 = col@row
    u_x_y_2[:,0,0] += 1
    u_x_y_2[:,1,1] += 1
    return u_x_y_2

# Auxiliary Functions


In [ ]:
def area(V,tris):
    v1,v2,v3 = V[tris[:,0]],V[tris[:,1]],V[tris[:,2]]
    return 0.5*torch.abs((v2[:,0]-v1[:,0])*(v3[:,1]-v1[:,1])-(v3[:,0]-v1[:,0])*(v2[:,1]-v1[:,1]))
    
def centroid(V,tris):
    return (V[tris[:,0]]+V[tris[:,1]]+V[tris[:,2]])/3.0

def Plot(V, U):
    _,ax=plt.subplots()
    ax.tripcolor(V[:,0],V[:,1],tris,U,cmap='viridis')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title('U')
    plt.show()
    
Plot(V,U(V))

# Neural Network

In [ ]:
class FCN(nn.Module):
    def __init__(self,layers):
        super().__init__()
        self.layers = layers 
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction ='mean')
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)]) 
        self.iter = 0
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            nn.init.zeros_(self.linears[i].bias.data)   

    def forward(self,x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        a = x.float()
        for i in range(len(self.layers)-2):  
            z = self.linears[i](a)              
            a = self.activation(z)
        a = self.linears[-1](a)
        return a    
    
    def jacobian(self,V):
        V.requires_grad = True
        xy = self.forward(V)
        x = xy[:,0].unsqueeze(1)
        y = xy[:,1].unsqueeze(1)
        dx = autograd.grad(x,V,torch.ones([V.shape[0],1]).to(device),retain_graph=True,create_graph=True)[0]
        dy = autograd.grad(y,V,torch.ones([V.shape[0],1]).to(device),retain_graph=True,create_graph=True)[0]
        return torch.stack([dx,dy],dim=1)
    
    def loss_BE(self,boundarys):
        return self.loss_function(self.forward(boundarys),boundarys)*1000.0
    
    def loss_G(self,V_ref,tris):
        areas = area(V_ref,tris)
        centroids = centroid(V_ref,tris)
        cx = self.forward(centroids)
        J = self.jacobian(centroids)
        detJ = torch.linalg.det(J)
        Jinv = torch.linalg.inv(J)
        JinvT = Jinv.transpose(1,2)
        M = metric(cx)
        M_inv = torch.linalg.inv(M)
        JMJ = Jinv@M_inv@JinvT
        traces = JMJ[:,0,0] + JMJ[:,1,1]  
        return self.loss_function(torch.sum(areas*traces*detJ),torch.tensor(0.0).to(device))

    def loss(self,V_ref,V,tris,boundarys):      
        return self.loss_G(V_ref,tris) + self.loss_BE(boundarys)
        
        # V.requires_grad = True
        # V_ref.requires_grad = True
        # J = self.jacobian(V_ref,V,tris)
        # JT = J.transpose(1,2)
        # vol = self.area(V,tris)
        # JJT = torch.bmm(J,JT)
        # traces = JJT[:,0,0] + JJT[:,1,1]
        # return torch.dot(vol, traces)

# Train Neural Network

In [ ]:
V = V.float().to(device)
V_ref = V_ref.float().to(device)
tris = torch.tensor(tris,dtype=torch.long).to(device)
boundarys = boundarys.float().to(device)

MMPDE_Net = FCN(layers)
MMPDE_Net.to(device)
params = list(MMPDE_Net.parameters())
optimizer = torch.optim.Adam(MMPDE_Net.parameters(),lr=lr,amsgrad=False)

for i in range(steps):
    loss = MMPDE_Net.loss(V_ref,V,tris,boundarys)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(i,loss.detach().cpu().numpy())
    
new_V = MMPDE_Net(V_ref).detach().cpu()
plt.triplot(new_V[:,0],new_V[:,1],tris.detach().cpu(),'k-',linewidth=0.3)